# Predizione sul sesso dei maratoneti di Boston
I dati si riferiscono alla maratona di Boston del 2016 e sono stati recuperato da [Kaggle](https://www.kaggle.com/rojour/boston-results).

# Utilizzo della libreria Pandas
La lettura e la manipolazione di dati risulta molto semplice usando la libreria [Pandas]().
Gli oggetti principali di questa libreria sono:

1. Le **Serie**
2. I **DataFrame**

Invece di dare prima una definizione formale di cosa sono, vediamo subito dei semplici esempio d'uso.

Prima ancora, vediamo come leggere un semplice file .CSV in Python, che lo memorizza direttamente in un *dataFrame*.

## Lettura dati da un file .CSV

In [ ]:
import pandas as pd

In [ ]:
# E` necessario convertire il tempo di gara in secondi, per poterlo confrontare nelle regressioni
bm = pd.read_csv('./data/marathon_results_2016.csv')
bm[:3]

In [ ]:
type(bm)

In [ ]:
bm[['Age', 'Official Time']][:3]

In [ ]:
bm.info()

In [ ]:
# Vorremmo predire il sesso (label) in base all'età e al tempo ufficiale (covariate)
bm = bm[['Age', 'M/F', 'Official Time']]
bm.info()

In [ ]:
ts = bm['Age']
print(type(ts))
print(ts[:4])

In [ ]:
def time_convert(x):
    """ Converti una stringa dal formato hh:mm:ss in numero
        in caso di dato non leggibile, restituire il numero 0"""
    try:
        # DA COMPLETARE
        # QUANDO QUESTO METODO VIENE INVOCATO, x È UNA STRINGA
    except:
        return float('0.0')

In [ ]:
# E` necessario convertire il tempo di gara in secondi, per poterlo confrontare nelle regressioni
bm = pd.read_csv('./data/marathon_results_2016.csv', converters={'Official Time': time_convert})
bm[-10:]

In [ ]:
from sklearn.model_selection import train_test_split

# Generate TRAINING e TEST sets
x_train, x_test, y_train, y_test = train_test_split(bm[['Age', 'Official Time', '5K']], list(map(lambda x: int(x == 'M'), bm['M/F'])), random_state=0)

In [ ]:
x_train[:3]

In [ ]:
y_train[:3]

## Analisi dei dati naive

In [ ]:
# Percentuale di uomini vs. donne che hanno concluso la gara
n_male = sum(y_train)
n_female = len(y_train) - n_male
perc_male = round(100*n_male/(n_male+n_female), 2)
perc_female = 100 - perc_male
print("Uomini: ", n_male, ", " , perc_male, "% - Donne: ", n_female, ", ", perc_female, "%", sep='')

In [ ]:
def ConstantModel(Xs, Y):
    # DA FARE: SCRIVERE UNA FUNZIONE CHE RESTITUISCA UNA FUNZIONE
    #          CHE PER OGNI PREDIZIONE RICHIESTA, PREDICE SEMPRE "male"
    pass

# Test il modello costante
LearnedModel = ConstantModel(x_train, y_train)
y_pred = LearnedModel(x_test)
# SI OSSERVINO LE MISURE FONDAMENTALI...
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
print(mean_absolute_error(y_test, y_pred), mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred))
# ... E LE ALTRE STATISTICHE VISTE A LEZIONE
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print('ACCURACY:',accuracy_score(y_test, y_pred))
print('REPORT:',classification_report(y_test, y_pred))
print('Confusion Matrix:', confusion_matrix(y_test, y_pred))

# Rappresentazioni grafiche con Seaborn

In [ ]:
import seaborn as sns

In [ ]:
sns.jointplot(data=bm, x='Age', y='5K', kind='reg', color='g')
sns.plt.show()

In [ ]:
bm_female = bm[bm['M/F'] == 'F']
sns.jointplot(data=bm_female, x='Age', y='5K', kind='reg', color='r')
sns.plt.show()

In [ ]:
bm_male = bm[bm['M/F'] == 'M']
sns.jointplot(data=bm_male, x='Age', y='5K', kind='reg', color='b')
sns.plt.show()

## Predizione tramite i *k* campioni più simili

Per una rapida descrizione del metodo si rimanda alla documentazione ufficiale:

1. Introduzione: [Nearest Neighbors](http://scikit-learn.org/stable/modules/neighbors.html) 
2. Metodo da usare: [KNeighborsClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)

**IMPORTANTE:** Per imparare a programmare, si deve imparare a leggere e studiare la **DOCUMENTAZIONE** delle librerie che si utilizzano!

**ESERCIZIO 1:** Provare il classificatore seguente provando a cambiare il numero di vicini utilizzati (parametro `n_neighbors`). Provare a usare due algoritmi diversi: `ball_tree` e `kd_tree` (leggere la documentazione per capire come fare). Commentare la differenza di risultati ottenuti con i due metodi.

In [ ]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors=5)
# Input to this function must be "DataFrames"
knn.fit(x_train, y_train)

In [ ]:
y_pred = knn.predict(x_test)
print(mean_absolute_error(y_test, y_pred), mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print('ACCURACY:',accuracy_score(y_test, y_pred))
print('REPORT:',classification_report(y_test, y_pred))
print('Confusion Matrix:', confusion_matrix(y_test, y_pred))

## Predizione con Regressione Lineare
Per un semplice esempio di come eseguire una regressione lineare con SciKit Learn, consultare l'esempio seguente sulla [predizione del diabete](http://scikit-learn.org/stable/tutorial/statistical_inference/supervised_learning.html)

**ESERCIZIO 2:** Usando la libreria SciKit, implementare una semplice regressione lineare per classificare il sesso di un corridore in base ad alcune delle sue covariate. **CONFRONTARE** le statistiche di base con il predittore "costante" e quello ottenuto tramite il metodo dei "k" campioni più simili (il `KNeighborsClassifier` usato sopra).